<a href="https://colab.research.google.com/github/wooheehee/deeplearning-practice/blob/main/CH7_MNIST_%EC%88%AB%EC%9E%90%EB%B6%84%EB%A5%98%EA%B8%B0_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test / 255.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

In [3]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)

In [4]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(1024, activation='relu')

    self.output_layer = tf.keras.layers.Dense(10, activation=None)

  def call(self, x):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = self.conv_layer_1(x_image)
    h_pool1 = self.pool_layer_1(h_conv1)
    h_conv2 = self.conv_layer_2(h_pool1)
    h_pool2 = self.pool_layer_2(h_conv2)
    h_pool2_flat = self.flatten_layer(h_pool2)
    h_fc1 = self.fc_layer_1(h_pool2_flat)
    logits = self.output_layer(h_fc1)
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

In [5]:
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [6]:
optimizer = tf.optimizers.Adam(1e-4)

In [10]:
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred, logits = model(x)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [7]:
@tf.function 
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [8]:
CNN_model = CNN()

In [12]:
for i in range(1000):
  batch_x, batch_y = next(train_data_iter)
  if i % 100 == 0:
    train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y)
    print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f" % (i, train_accuracy))
  train_step(CNN_model, batch_x, batch_y)

반복(Epoch): 0, 트레이닝 데이터 정확도: 0.940000
반복(Epoch): 100, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 200, 트레이닝 데이터 정확도: 0.940000
반복(Epoch): 300, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 400, 트레이닝 데이터 정확도: 0.940000
반복(Epoch): 500, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 600, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 700, 트레이닝 데이터 정확도: 0.960000
반복(Epoch): 800, 트레이닝 데이터 정확도: 0.940000
반복(Epoch): 900, 트레이닝 데이터 정확도: 0.920000


In [13]:
print("정확도(Accuracy): %f" % compute_accuracy(CNN_model(x_test)[0], y_test))

정확도(Accuracy): 0.971500
